# 🚀 DeepTrust Model Training

Use this notebook to train your deepfake detection model using a free GPU.
Supports both image-based (CIFAKE) and video-based (FaceForensics++) datasets.

### Steps:
1. Setup Environment
2. Download Data (CIFAKE or FaceForensics++)
3. Train Model
4. Download Trained Weights

In [ ]:
# @title 1. Setup Environment
!options="ND"
# Check GPU
!nvidia-smi

# Clone/Update Repository
import os
if os.path.exists("deepfake-analyzer"):
    %cd deepfake-analyzer
    !git pull
else:
    !git clone https://github.com/Abhishek-Jose7/deepfake-analyzer.git
    %cd deepfake-analyzer

# Install Dependencies
!pip install -r requirements.txt
!pip install kagglehub

In [ ]:
# @title 2. Download Dataset (FIXED)
import kagglehub
import shutil
from pathlib import Path

DATASET = "CIFAKE"  # @param ["CIFAKE", "FaceForensics"]

if DATASET == "CIFAKE":
    print("⬇️ Downloading CIFAKE...")
    source = Path(
        kagglehub.dataset_download(
            "birdy654/cifake-real-and-ai-generated-synthetic-images"
        )
    )

    target = Path("dataset")
    if target.exists():
        shutil.rmtree(target)

    (target / "train" / "real").mkdir(parents=True)
    (target / "train" / "fake").mkdir(parents=True)
    (target / "val" / "real").mkdir(parents=True)
    (target / "val" / "fake").mkdir(parents=True)

    shutil.copytree(source / "train" / "REAL", target / "train" / "real", dirs_exist_ok=True)
    shutil.copytree(source / "train" / "FAKE", target / "train" / "fake", dirs_exist_ok=True)
    shutil.copytree(source / "test"  / "REAL", target / "val"   / "real", dirs_exist_ok=True)
    shutil.copytree(source / "test"  / "FAKE", target / "val"   / "fake", dirs_exist_ok=True)

    print("✅ CIFAKE normalized at ./dataset")

elif DATASET == "FaceForensics":
    print("⬇️ Downloading FaceForensics++...")
    path = kagglehub.dataset_download("hungle3401/faceforensics")
    shutil.copytree(path, "dataset_ff", dirs_exist_ok=True)
    print("✅ FaceForensics Ready at ./dataset_ff")

In [ ]:
# @title 3. Start Training
EPOCHS = 10 # @param {type:"slider", min:1, max:50, step:1}
BATCH_SIZE = 32 # @param {type:"integer"}
BACKBONE = "efficientnet_b0" # @param ["efficientnet_b0", "efficientnet_b3", "resnet18"]

dataset_path = "dataset" if DATASET == "CIFAKE" else "dataset_ff"
# CIFAKE uses generic ImageDataset, FaceForensics uses faceforensics Dataset
dataset_type = "generic" if DATASET == "CIFAKE" else "faceforensics"

# Sanity Check
if DATASET == "CIFAKE":
    !ls dataset/train
    !ls dataset/train/real | head
    !ls dataset/train/fake | head

!python train.py --dataset {dataset_path} --dataset-type {dataset_type} --epochs {EPOCHS} --batch-size {BATCH_SIZE} --backbone {BACKBONE}

In [ ]:
# @title 4. Download Trained Model
from google.colab import files
import os

if os.path.exists("checkpoints/best.pth"):
    print("Downloading best model...")
    files.download("checkpoints/best.pth")
else:
    print("⚠️ No model found! Did training finish?")